# Django Helper: Solr Queries
This is a general purpose Django Helper set up to test and write different functions to help with Solr queries. The output of these queries will be presented in Django based framework. 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import sqlalchemy
import sklearn
import time
import os
import pprint
import json
import urllib.request
from pprint import pprint
from datetime import datetime
from sqlalchemy import create_engine
from scipy import stats
from numpy import newaxis
from tensorflow.contrib.layers import fully_connected
from django.core.paginator import EmptyPage, PageNotAnInteger, Paginator
from django.shortcuts import render
from django.http import HttpResponse
from django.http import Http404
from django.shortcuts import render
from django.template import loader
from os.path import isdir
from os.path import abspath
from urllib import *
from collections import defaultdict

C:\Users\pshar\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


SOLR URL and Collection name

In [2]:
SOLR_BASE_URL = "http://localhost:8984/solr"
collection = "markets3"

Setup SOLR query Let us start with seting up SOLR for NoSQL along with a sample query.

In [3]:
# listWords: List of words that are to be search
def querySolr(SOLR_BASE_URL, collection, listWords):
    searchPattern = ""
    grpStatus = "true"
    groupFld = "sentHash"
    words = listWords.split()
    for word in words:
        searchPattern += ('%2B')+word
    print(searchPattern) 
    
    url = SOLR_BASE_URL+"/"+collection+"/select?defType=dismax&q="+searchPattern+"&fl=*,score&qf=sentence^10.0+word^1.0+lemma^1.0&group="+grpStatus+"&group%2Efield="+groupFld+"&rows=100&start=0"
    print(url)
    connection = urllib.request.urlopen(url)
    httpResponse = json.load(connection)
    return httpResponse

Provide the input text as a list of words and call the function to execute query. Finally, check the HTTP response in JSON format. 

In [4]:
listWords = "risk ccar"
httpResponse = querySolr(SOLR_BASE_URL, collection, listWords)
httpResponse

%2Brisk%2Bccar
http://localhost:8984/solr/markets3/select?defType=dismax&q=%2Brisk%2Bccar&fl=*,score&qf=sentence^10.0+word^1.0+lemma^1.0&group=true&group%2Efield=sentHash&rows=100&start=0


{'grouped': {'sentHash': {'groups': [{'doclist': {'docs': [{'_version_': 1629725295040790528,
        'fileName': 'ge10k2018htm',
        'filePath': 'C:/Users/pshar/Dropbox/WebServices/mysite/polls/templates/polls/FilingsBySymbols\\\\GE\\\\10-K\\\\ge10k2018htm',
        'filing': '10-K',
        'id': '958d049e-5c3a-412d-8019-2c5a4919e2da',
        'lemma': ['ge',
         '2018',
         'form',
         '10-k',
         '78',
         'ge',
         '2018',
         'form',
         '10-k',
         '78',
         'risk',
         'factor',
         'risk',
         'factor',
         'risk',
         'factor',
         'risk',
         'factor',
         'risk',
         'factor',
         'the',
         'follow',
         'discuss',
         'risk',
         'factor',
         'contain',
         '``',
         'forward-look',
         'statement',
         '``',
         'discuss',
         'forward-look',
         'statement',
         'section'],
        'ner': ['ner',
      

In [5]:
count = 0
data = httpResponse['grouped']['sentHash']['groups']
for item in data:
    count += 1
count

100

In [6]:
i = 0
#while (i < count):
for each in httpResponse['grouped']['sentHash']['groups']:
    record = httpResponse['grouped']['sentHash']['groups'][i]['doclist']['docs'][0]
    print(record['sentence'])
    print(record['fileName'])
    print(record['symbol'])
    print(record['filing'])
    print(record['score'])
    i += 1
    #httpResponse['grouped']['sentHash']['groups'][i]['doclist']['docs']['maxScore']

GE 2018 FORM 10-K 78 GE 2018 FORM 10-K 78 RISK FACTORS RISK FACTORS RISK FACTORS RISK FACTORS RISK FACTORS The following discussion of risk factors contains `` forward-looking statements '' as discussed in the Forward-Looking Statements section
ge10k2018htm
GE
10-K
66.8615
RISK MANAGERS Risk assessment and risk management are the responsibility of management and are carried out through risk managers who are operationally integrated into each of our businesses
ge10k2016htm
GE
10-K
65.09168
GE 2018 FORM 10-K 81 GE 2018 FORM 10-K 81 RISK FACTORS RISK FACTORS RISK FACTORS RISK FACTORS Cybersecurity Increased cybersecurity requirements vulnerabilities threats and more sophisticated and targeted computer crime could pose a risk to our systems networks products solutions services and data
ge10k2018htm
GE
10-K
63.235744
GE 2017 FORM 10-K 105 GE 2017 FORM 10-K 105 RISK FACTORS RISK FACTORS RISK FACTORS RISK FACTORS RISK FACTORS The following discussion of risk factors contains `` forward-lookin

Check count of results returned by query

In [7]:
#def queryForCoreSearch(words):
count = httpResponse['grouped']['sentHash']['matches']
if (count > 100):
    count = 100
count

100

Define a defaultdict that will record each resuch with an int as "key" and a list of [symbol, fileName, sentence, word] as "value".

In [8]:
entries = defaultdict(list)
entries

defaultdict(list, {})

Now, parse the query's JSON response and retrieve contents in the defaultdict just created.

In [9]:
i = 0
    
for each in httpResponse['response']['docs']:
    single_result = []   
    symbol = (each['symbol'][0]).replace("\'", "")
    fileName = each['fileName'][0].replace("'", "")
    score = each['score']
    single_result.append(symbol)
    single_result.append(fileName)
    if ('word' in each):
        word = each['word'][0].replace("'", "")
        single_result.append(word)
    elif ('sentence' in each):
        sentence = each['sentence'][0].replace("'", "")
        single_result.append(sentence)
    single_result.append(score)
    
    if (i < count):
        entries[i].append(single_result)
        i = i+1

KeyError: 'response'

Here is what we got as plain-test result:

In [ ]:
entries

In [ ]:
# After much pain I found out that Django doesnt handle defaultdict in template library. Need to convert it into plain dict
# Reference: https://stackoverflow.com/questions/32813500/items-not-working-on-defaultdict-in-django-template.
dictEntries = dict(entries)
dictEntries

Now finally print the entries.

In [ ]:
for key, value in dictEntries.items():
    for singleItemList in value:
        for item in singleItemList:
            print(item)